In [1]:
import numpy as np
import cv2
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# Keypoints using Mediapipe

Here we check if we have our web cam access. Then we add an addition layer showing keypoints in our body

In [3]:
mp_holistics = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Layer drawing utilities

Create a function so that our while loop does not get cluttered. We pass a image frame and a model, this model processes the frame and detects the keypoints found. This function then returns the frame and keypoints

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    result = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, result

Function to draw the detected landmarks upon a given frame

In [10]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistics.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistics.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistics.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistics.HAND_CONNECTIONS)

In [30]:
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistics.FACEMESH_TESSELATION, 
                              None,
                              mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=1, circle_radius=1)
                             )
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistics.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=2),
                              mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=1, circle_radius=1)
                             )
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistics.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=3, circle_radius=3),
                              mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=1, circle_radius=1)
                             )
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistics.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=3, circle_radius=3),
                              mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=1, circle_radius=1)
                             )

In [65]:
cap = cv2.VideoCapture(0)

# Initialise mediapipe model
with mp_holistics.Holistic(min_detection_confidence=0.7, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read our feed
        ret, frame = cap.read()

        # Perform keypoint detection
        image, result = mediapipe_detection(frame, holistic)
        
        # Draw landmarks on the frame
        draw_styled_landmarks(image, result)

        # Show to screen ('title', frame)
        cv2.imshow('OpenCV feed', image)

        # Catch event when we want to quit (q)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    # Cleanup
    cap.release()
    cv2.destroyAllWindows()

# Extract Keypoint values

Extracting and processing keypoint values into a format that we can use

In [68]:
result.pose_landmarks.landmark

[x: 0.4553624987602234
y: 0.5731104016304016
z: -1.2322654724121094
visibility: 0.9997774362564087
, x: 0.48357701301574707
y: 0.48920854926109314
z: -1.185476541519165
visibility: 0.9994886517524719
, x: 0.5012804865837097
y: 0.48758044838905334
z: -1.1852549314498901
visibility: 0.9995299577713013
, x: 0.5211310982704163
y: 0.48568785190582275
z: -1.1856610774993896
visibility: 0.9994864463806152
, x: 0.41371744871139526
y: 0.4903739094734192
z: -1.1987465620040894
visibility: 0.9995215535163879
, x: 0.387626975774765
y: 0.49075374007225037
z: -1.19772207736969
visibility: 0.9994756579399109
, x: 0.3643929064273834
y: 0.492742121219635
z: -1.197809100151062
visibility: 0.9994405508041382
, x: 0.5464252233505249
y: 0.5014891028404236
z: -0.7554653286933899
visibility: 0.9996281266212463
, x: 0.3289526104927063
y: 0.513339638710022
z: -0.7723501920700073
visibility: 0.999731183052063
, x: 0.4904845654964447
y: 0.6480852365493774
z: -1.0467863082885742
visibility: 0.9998200535774231
, x

In [75]:
def extract_keypoints(result):
    pose = np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in result.pose_landmarks.landmark]).flatten() if result.face_landmarks else np.zeros(33*4)
    face = np.array([[landmark.x, landmark.y, landmark.z] for landmark in result.face_landmarks.landmark]).flatten() if result.face_landmarks else np.zeros(468*3)
    left_hand = np.array([[landmark.x, landmark.y, landmark.z] for landmark in result.left_hand_landmarks.landmark]).flatten() if result.left_hand_landmarks else np.zeros(21*3)
    right_hand = np.array([[landmark.x, landmark.y, landmark.z] for landmark in result.right_hand_landmarks.landmark]).flatten() if result.right_hand_landmarks else np.zeros(21*3)
    
    return np.concatenate([pose, face, left_hand, right_hand])

In [76]:
result_test = extract_keypoints(result)


In [77]:
print(result_test)

[ 0.4553625   0.5731104  -1.23226547 ...  0.          0.
  0.        ]


# Setup folders for data collection

In [46]:
os.path

<module 'ntpath' from 'C:\\Users\\LENOVO\\anaconda3\\envs\\tensorflow\\lib\\ntpath.py'>

In [51]:
# Path for exported data (numpy arrays)
DATA_PATH = os.path.join('Keypoint Data')

# Actions that we try to detect
# actions = np.array(['Hello', 'Thanks', 'I Love You'])
actions = np.array(['Hello'])
no_sequences = 10
sequence_length = 30

In [82]:
for action in actions:
    for seq in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(seq)))
        except:
            pass

# Collect keypoint data for Training and Testing

Collect live feed and store it in the created folders.

Collect a sample for each action.

In [109]:
cap = cv2.VideoCapture(0)

# Initialise mediapipe model
with mp_holistics.Holistic(min_detection_confidence=0.7, min_tracking_confidence=0.5) as holistic:
    # Loop each action
    for action in actions:
        # Collect a defined no of sample videos
        for seq in range(no_sequences):
            # Collect defined no of frames per video
            for frame_no in range(sequence_length):

                # Read our feed
                ret, frame = cap.read()

                # Perform keypoint detection
                image, result = mediapipe_detection(frame, holistic)

                # Draw landmarks on the frame
                draw_styled_landmarks(image, result)
                
                # Apply wait logic
                if frame_no == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4 ,cv2.LINE_AA)
                    cv2.putText(image, 'Collecting Video {} for {}'.format(seq, action), (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1 ,cv2.LINE_AA)
                    # Show to screen ('title', frame)
                    cv2.imshow('OpenCV feed', image)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image, 'Collecting Video {} for {}'.format(seq, action), (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1 ,cv2.LINE_AA)
                    # Show to screen ('title', frame)
                    cv2.imshow('OpenCV feed', image)
                    
                # Export keypoints to folders
                keypoints = extract_keypoints(result)
                npy_path = os.path.join(DATA_PATH, action, str(seq), str(frame_no))
                np.save(npy_path, keypoints)

                # Catch event when we want to quit (q)
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

    # Cleanup
    cap.release()
    cv2.destroyAllWindows()

In [81]:
cap.release()
cv2.destroyAllWindows()

# Preprocess data and Create labels and features

Load saved data.
Create labels for the data.
Perform the train test split.

In [55]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [56]:
label_map = {label:num for num, label in  enumerate(actions)}

In [57]:
label_map

{'Hello': 0, 'Thanks': 1, 'I Love You': 2}

In [110]:
sequences, labels = [], []
for action in actions:
    for seq in range(no_sequences):
        window = []
        for frame_no in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(seq), "{}.npy".format(frame_no)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [111]:
X = np.array(sequences)
X.shape

(30, 30, 1662)

In [112]:
y = to_categorical(labels).astype(int)
y

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]])

In [113]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

# Build and Train an LSTM model

An lstm(long shot term memory) model is majorly used in the Natural language processing area of MAchine Learning. This model has a (psuedo) memory that can remember things that are important and forgets the not so important things.

In [88]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [114]:
log_dir = os.path.join('logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [115]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [116]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            442112    
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-tr

In [117]:
X.shape

(30, 30, 1662)

In [118]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [119]:
model.fit(X_train, y_train, epochs=500, callbacks=[tb_callback])

Epoch 1/500
1/1 [==============================] - 3s 3s/step - loss: 1.1142 - categorical_accuracy: 0.4444
Epoch 2/500
1/1 [==============================] - 0s 66ms/step - loss: 1.1294 - categorical_accuracy: 0.3333
Epoch 3/500
1/1 [==============================] - 0s 61ms/step - loss: 1.1096 - categorical_accuracy: 0.3704
Epoch 4/500
1/1 [==============================] - 0s 64ms/step - loss: 1.1060 - categorical_accuracy: 0.2593
Epoch 5/500
1/1 [==============================] - 0s 58ms/step - loss: 1.1166 - categorical_accuracy: 0.3333
Epoch 6/500
1/1 [==============================] - 0s 62ms/step - loss: 1.0977 - categorical_accuracy: 0.3333
Epoch 7/500
1/1 [==============================] - 0s 63ms/step - loss: 1.0937 - categorical_accuracy: 0.4074
Epoch 8/500
1/1 [==============================] - 0s 62ms/step - loss: 1.0984 - categorical_accuracy: 0.2593
Epoch 9/500
1/1 [==============================] - 0s 59ms/step - loss: 1.1459 - categorical_accuracy: 0.2593
Epoch 10/500

1/1 [==============================] - 0s 60ms/step - loss: 1.0532 - categorical_accuracy: 0.3333
Epoch 75/500
1/1 [==============================] - 0s 63ms/step - loss: 1.0327 - categorical_accuracy: 0.5926
Epoch 76/500
1/1 [==============================] - 0s 62ms/step - loss: 0.9913 - categorical_accuracy: 0.6296
Epoch 77/500
1/1 [==============================] - 0s 61ms/step - loss: 0.8898 - categorical_accuracy: 0.4815
Epoch 78/500
1/1 [==============================] - 0s 62ms/step - loss: 0.8138 - categorical_accuracy: 0.4444
Epoch 79/500
1/1 [==============================] - 0s 62ms/step - loss: 1.0959 - categorical_accuracy: 0.5185
Epoch 80/500
1/1 [==============================] - 0s 64ms/step - loss: 0.6299 - categorical_accuracy: 0.6296
Epoch 81/500
1/1 [==============================] - 0s 62ms/step - loss: 1.7777 - categorical_accuracy: 0.3704
Epoch 82/500
1/1 [==============================] - 0s 61ms/step - loss: 0.7967 - categorical_accuracy: 0.5556
Epoch 83/500
1

1/1 [==============================] - 0s 61ms/step - loss: 0.3793 - categorical_accuracy: 0.8889
Epoch 148/500
1/1 [==============================] - 0s 59ms/step - loss: 0.5427 - categorical_accuracy: 0.6667
Epoch 149/500
1/1 [==============================] - 0s 59ms/step - loss: 0.4698 - categorical_accuracy: 0.7778
Epoch 150/500
1/1 [==============================] - 0s 60ms/step - loss: 0.3312 - categorical_accuracy: 0.8889
Epoch 151/500
1/1 [==============================] - 0s 61ms/step - loss: 0.4122 - categorical_accuracy: 0.6667
Epoch 152/500
1/1 [==============================] - 0s 63ms/step - loss: 0.3146 - categorical_accuracy: 0.8519
Epoch 153/500
1/1 [==============================] - 0s 58ms/step - loss: 0.4237 - categorical_accuracy: 0.8889
Epoch 154/500
1/1 [==============================] - 0s 60ms/step - loss: 0.3276 - categorical_accuracy: 0.8519
Epoch 155/500
1/1 [==============================] - 0s 63ms/step - loss: 0.4721 - categorical_accuracy: 0.7407
Epoch 

1/1 [==============================] - 0s 63ms/step - loss: 0.2573 - categorical_accuracy: 0.8889
Epoch 221/500
1/1 [==============================] - 0s 67ms/step - loss: 0.2538 - categorical_accuracy: 0.8889
Epoch 222/500
1/1 [==============================] - 0s 63ms/step - loss: 0.2566 - categorical_accuracy: 0.8889
Epoch 223/500
1/1 [==============================] - 0s 65ms/step - loss: 0.2546 - categorical_accuracy: 0.8889
Epoch 224/500
1/1 [==============================] - 0s 65ms/step - loss: 0.2432 - categorical_accuracy: 0.8889
Epoch 225/500
1/1 [==============================] - 0s 63ms/step - loss: 0.2551 - categorical_accuracy: 0.8889
Epoch 226/500
1/1 [==============================] - 0s 64ms/step - loss: 0.2406 - categorical_accuracy: 0.8889
Epoch 227/500
1/1 [==============================] - 0s 61ms/step - loss: 0.2448 - categorical_accuracy: 0.8889
Epoch 228/500
1/1 [==============================] - 0s 61ms/step - loss: 0.2396 - categorical_accuracy: 0.8889
Epoch 

1/1 [==============================] - 0s 60ms/step - loss: 0.2504 - categorical_accuracy: 0.8519
Epoch 294/500
1/1 [==============================] - 0s 62ms/step - loss: 0.2605 - categorical_accuracy: 0.8519
Epoch 295/500
1/1 [==============================] - 0s 61ms/step - loss: 0.2296 - categorical_accuracy: 0.8889
Epoch 296/500
1/1 [==============================] - 0s 61ms/step - loss: 0.2205 - categorical_accuracy: 0.8889
Epoch 297/500
1/1 [==============================] - 0s 64ms/step - loss: 0.2396 - categorical_accuracy: 0.8519
Epoch 298/500
1/1 [==============================] - 0s 60ms/step - loss: 0.2428 - categorical_accuracy: 0.8889
Epoch 299/500
1/1 [==============================] - 0s 58ms/step - loss: 0.2185 - categorical_accuracy: 0.8519
Epoch 300/500
1/1 [==============================] - 0s 57ms/step - loss: 0.2105 - categorical_accuracy: 0.8889
Epoch 301/500
1/1 [==============================] - 0s 57ms/step - loss: 0.2251 - categorical_accuracy: 0.8889
Epoch 

1/1 [==============================] - 0s 59ms/step - loss: 0.6090 - categorical_accuracy: 0.8519
Epoch 367/500
1/1 [==============================] - 0s 56ms/step - loss: 0.6119 - categorical_accuracy: 0.8519
Epoch 368/500
1/1 [==============================] - 0s 59ms/step - loss: 0.6076 - categorical_accuracy: 0.8519
Epoch 369/500
1/1 [==============================] - 0s 61ms/step - loss: 0.5965 - categorical_accuracy: 0.8519
Epoch 370/500
1/1 [==============================] - 0s 64ms/step - loss: 0.5798 - categorical_accuracy: 0.8148
Epoch 371/500
1/1 [==============================] - 0s 59ms/step - loss: 0.5622 - categorical_accuracy: 0.8519
Epoch 372/500
1/1 [==============================] - 0s 58ms/step - loss: 0.5475 - categorical_accuracy: 0.8519
Epoch 373/500
1/1 [==============================] - 0s 62ms/step - loss: 0.5395 - categorical_accuracy: 0.9259
Epoch 374/500
1/1 [==============================] - 0s 59ms/step - loss: 0.5359 - categorical_accuracy: 0.8519
Epoch 

1/1 [==============================] - 0s 62ms/step - loss: 0.1220 - categorical_accuracy: 0.9630
Epoch 440/500
1/1 [==============================] - 0s 61ms/step - loss: 0.1155 - categorical_accuracy: 0.8889
Epoch 441/500
1/1 [==============================] - 0s 60ms/step - loss: 0.1135 - categorical_accuracy: 0.9259
Epoch 442/500
1/1 [==============================] - 0s 60ms/step - loss: 0.1084 - categorical_accuracy: 0.9259
Epoch 443/500
1/1 [==============================] - 0s 60ms/step - loss: 0.1472 - categorical_accuracy: 0.8889
Epoch 444/500
1/1 [==============================] - 0s 60ms/step - loss: 0.1366 - categorical_accuracy: 0.9259
Epoch 445/500
1/1 [==============================] - 0s 62ms/step - loss: 0.1460 - categorical_accuracy: 0.9259
Epoch 446/500
1/1 [==============================] - 0s 63ms/step - loss: 0.1376 - categorical_accuracy: 0.9630
Epoch 447/500
1/1 [==============================] - 0s 59ms/step - loss: 0.1581 - categorical_accuracy: 0.8889
Epoch 

# Make predictions and test model accuracy

In [130]:
res = model.predict(X_test)

1/1 [==============================] - 0s 230ms/step


In [97]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [132]:
y_pred = np.argmax(res, axis=1).tolist()
y_true = np.argmax(y_test, axis=1).tolist()

In [133]:
multilabel_confusion_matrix(y_true, y_pred)

array([[[2, 0],
        [0, 1]],

       [[2, 0],
        [0, 1]],

       [[2, 0],
        [0, 1]]], dtype=int64)

In [134]:
accuracy_score(y_true, y_pred)

1.0

Save Model weights

In [ ]:
model.save('actions.h5')
del model

In [129]:
import tensorflow as tf
model = tf.keras.models.load_model('actions.h5')

# Testing in real time

Restructure our loop so that now we get real time predications

In [151]:
# Detection variables
sequence = []
sentence = []
predications = []
threshold = 0.75

In [147]:
colors = [(245,117,16), (117, 245, 16), (16, 117,245)]
def prob_show(res, action, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0, 60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
    return output_frame

In [153]:
cap = cv2.VideoCapture(0)

# Initialise mediapipe model
with mp_holistics.Holistic(min_detection_confidence=0.7, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read our feed
        ret, frame = cap.read()

        # Perform keypoint detection
        image, result = mediapipe_detection(frame, holistic)
        
        # Draw landmarks on the frame
        draw_styled_landmarks(image, result)
        
        # Prediction logic
        keypoints = extract_keypoints(result)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predications.append(np.argmax(res))
            
        # Visualisation logic
            if np.unique(predications[-10:])[0] == np.argmax(res):
                if res[np.argmax(res)] > threshold:
                    if len(sentence) > 0:
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5:
                sentence = sentence[-5:]

            image = prob_show(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

        # Show to screen ('title', frame)
        cv2.imshow('OpenCV feed', image)

        # Catch event when we want to quit (q)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    # Cleanup
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 15ms/step
Hello
1/1 [==============================] - 0s 15ms/step
Hello
1/1 [==============================] - 0s 14ms/step
Hello
1/1 [==============================] - 0s 14ms/step
Hello
1/1 [==============================] - 0s 14ms/step
Hello
1/1 [==============================] - 0s 15ms/step
Hello
1/1 [==============================] - 0s 15ms/step
Hello
1/1 [==============================] - 0s 15ms/step
Hello
1/1 [==============================] - 0s 15ms/step
Hello
1/1 [==============================] - 0s 16ms/step
Hello
1/1 [==============================] - 0s 15ms/step
Hello
1/1 [==============================] - 0s 14ms/step
Hello
1/1 [==============================] - 0s 14ms/step
Hello
1/1 [==============================] - 0s 16ms/step
Hello
1/1 [==============================] - 0s 14ms/step
Hello
1/1 [==============================] - 0s 14ms/step
Hello
1/1 [==============================] - 0s 24ms/step
Hello
1/1 [=========

1/1 [==============================] - 0s 15ms/step
Thanks
1/1 [==============================] - 0s 15ms/step
Thanks
1/1 [==============================] - 0s 14ms/step
Thanks
1/1 [==============================] - 0s 15ms/step
Thanks
1/1 [==============================] - 0s 15ms/step
Thanks
1/1 [==============================] - 0s 14ms/step
Thanks
1/1 [==============================] - 0s 13ms/step
I Love You
1/1 [==============================] - 0s 15ms/step
I Love You
1/1 [==============================] - 0s 16ms/step
I Love You
1/1 [==============================] - 0s 16ms/step
I Love You
1/1 [==============================] - 0s 19ms/step
I Love You
1/1 [==============================] - 0s 18ms/step
I Love You
1/1 [==============================] - 0s 16ms/step
I Love You
1/1 [==============================] - 0s 15ms/step
I Love You
1/1 [==============================] - 0s 15ms/step
I Love You
1/1 [==============================] - 0s 14ms/step
I Love You
1/1 [===========

1/1 [==============================] - 0s 14ms/step
Thanks
1/1 [==============================] - 0s 15ms/step
Thanks
1/1 [==============================] - 0s 15ms/step
Thanks
1/1 [==============================] - 0s 14ms/step
Thanks
1/1 [==============================] - 0s 15ms/step
Thanks
1/1 [==============================] - 0s 14ms/step
Thanks
1/1 [==============================] - 0s 15ms/step
Thanks
1/1 [==============================] - 0s 14ms/step
Thanks
1/1 [==============================] - 0s 20ms/step
Thanks
